# Operaciones básicas

Trataremos aquí las operaciones más básicas que se pueden realizar sobre las estructuras de datos de pandas. Estas operaciones tienen un funcionamiento prácticamente idéntico en Series y DataFrames. En caso de que esto no sea así en algún caso concreto se indicará explícitamente.

In [1]:
import numpy as np
import pandas as pd

## Tratamiento de Series y DataFrames como diccionarios

Dado que internamente tanto las Series como los DataFrames pueden verse como diccionarios, podemos apilcar sobre los mismos cualquier funcionalidad que aplicaríamos sobre diccionarios básicos del core de Python.<br/>
<b>IMPORTANTE:</b> Hay que tener en cuenta que en DataFrames el diccionario es un diccionario de "columnas".

In [2]:
serie = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [3]:
dataframe = pd.DataFrame({'var1': serie, 'var2': serie})
dataframe

,var1,var2
a,1,1
b,2,2
c,3,3
d,4,4


#### Indexación por clave

In [4]:
# Indexación mediante clave del índice en series
serie['a']

1

In [5]:
# Indexación por nombre de columna en dataframes
dataframe['var2']

a    1
b    2
c    3
d    4
Name: var2, dtype: int64

#### Comprobación de la existencia de una clave

In [6]:
# Comprobación de la existencia de una clave en el índice en series
'b' in serie

True

In [7]:
# Comprobación de la existencia de un nombre de columna en dataframes
'b' in dataframe

False

In [8]:
# Comprobación de la existencia de un nombre de columna en dataframes
'var1' in dataframe

True

#### Adición de elementos

<b>IMPORTANTE:</b> Al añadir columnas a un DataFrame, el tamaño del vector añadido deberá coincidir con el del DataFrame original. En caso contrario se recibirá un error.

In [9]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [10]:
# Adición de elementos a series
serie['e'] = 5
serie

a    1
b    2
c    3
d    4
e    5
dtype: int64

In [13]:
# Adición de elementos a dataframes
dataframe['var3'] = [5, 6, 7, 8]
dataframe

,var1,var2,var3
a,1,1,5
b,2,2,6
c,3,3,7
d,4,4,8


#### Eliminación de elementos

In [14]:
# Eliminación de elementos en series
del serie['e']
serie

a    1
b    2
c    3
d    4
dtype: int64

In [15]:
# Eliminación de columnas en dataframes
del dataframe['var3']
dataframe

,var1,var2
a,1,1
b,2,2
c,3,3
d,4,4


## Tratamiento de Series y DataFrames como ndarrays

Dado que, internamente, cualquier estructura de pandas está implementada sobre ndarrays de NumPy, es posible realizar sobre Series y DataFrames todas las operaciones que se pueden realizar sobre un ndarrays.<br/>

<b>IMPORTANTE:</b> Dado que un ndarray no puede mezclar elementos de diferentes tipos y un DataFrame sí, algunas de las operaciones sobre DataFrames estarán supeditadas a que todas sus columnas tengan el mismo tipo.

In [16]:
serie = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [19]:
dataframe = pd.DataFrame({'var1': pd.Series(serie, dtype=np.int32), 'var2': pd.Series(serie, dtype=np.float)}) #con np.string_ si funciona
dataframe

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

#### Consulta de la composición

Disponemos de los mismos atributos de consulta que en ndarrays, si bien hay que tener en cuenta que:<br/>
<ul>
<li>El atributo <b>dtype</b> será <b>dtypes</b> en DataFrames dada la posibilidad de múltiples tipos.</li>
<li>El atributo <b>ndim</b> en Series siempre valdrá 1 dado que siempre son estructuras unidimensionales y 2 en DataFrames dado que siempre son estructuras bidimensionales.</li>
</ul>

In [20]:
# Consulta del tipo almacenado en una serie
serie.dtype

dtype('int64')

In [21]:
# Consulta de los tipos almacenados en un dataframe
dataframe.dtypes

var1     int32
var2    object
dtype: object

In [22]:
# Consulta del número de dimensiones en una serie
serie.ndim

1

In [23]:
# Consulta del número de dimensiones en un dataframe
dataframe.ndim

2

In [24]:
# Consulta de la forma de una serie
serie.shape

(4,)

In [25]:
# Consulta de la forma de un dataframe
dataframe.shape

(4, 2)

In [26]:
# Consulta del número de elementos de una serie
serie.size

4

In [27]:
# Consulta del número de elementos de un dataframe
dataframe.size

8

#### Operaciones con escalares

Al aplicar una operación sobre una estructura de pandas y un escalar se obtendrá otra estructura de pandas de idénticas características a la inicial pero con la operación aplicada elemento a elemento, <b>manteniendo el índice inalterado</b>.<br/>

<b>IMPORTANTE</b>: Dado que un DataFrame puede mezclar tipos muy diferentes en sus columnas, la aplicación de una operación con un escalar elemento a elemento puede no ser válida (p.e. operaciones matemáticas sobre cadenas).

In [28]:
# Suma de series y escalar
serie + 2

a    3
b    4
c    5
d    6
dtype: int64

In [29]:
# División de series y escalar
1 / serie

a    1.000000
b    0.500000
c    0.333333
d    0.250000
dtype: float64

In [30]:
# Multiplicación de dataframe y escalar
dataframe * 2

,var1,var2
a,2,b'11'
b,4,b'22'
c,6,b'33'
d,8,b'44'


In [31]:
# División de dataframe y escalar
1 / dataframe.var1

a    1.000000
b    0.500000
c    0.333333
d    0.250000
Name: var1, dtype: float64

#### Operaciones entre estructuras de pandas

Al aplicar una operación entre estructuras de pandas se aplicará la misma elemento a elemento. En el caso de pandas no es necesario, como lo era en NumPy, que los operandos tengan el mismo tamaño y forma ya que se aplicará un proceso de "alineación". Este proceso devolverá:<br/>
<ul>
<li>Como índices: la unión de las claves de ambos operandos.</li>
<li>Como valores: el resultado de aplicar la operación entre cada pareja de elementos (si coinciden las claves entre ambos operandos) o NaN (en caso contrario).</li>
</ul>

<b>IMPORTANTE:</b> De nuevo, el hecho de que un DataFrame pueda mezclar tipos en sus contenidos hace que no todas las operaciones matemáticas se puedan aplicar a los mismos.

In [32]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [33]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [34]:
serie1 = serie[:]
serie1

a    1
b    2
c    3
d    4
dtype: int64

In [35]:
serie1['e'] = 7
serie1

a    1
b    2
c    3
d    4
e    7
dtype: int64

In [36]:
dataframe1 = dataframe.copy() # Si no usamos copy, lo copia pero los dos dataframe se modifican
dataframe1['var3'] = [1, 2, 3, 4]
dataframe1

,var1,var2,var3
a,1,b'1',1
b,2,b'2',2
c,3,b'3',3
d,4,b'4',4


In [37]:
# Suma de series
serie + serie1

a    2.0
b    4.0
c    6.0
d    8.0
e    NaN
dtype: float64

In [38]:
# Suma de dataframes
dataframe + dataframe1

,var1,var2,var3
a,2,b'11',NaN
b,4,b'22',NaN
c,6,b'33',NaN
d,8,b'44',NaN


In [42]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [40]:
dataframe1

,var1,var2,var3
a,1,b'1',1
b,2,b'2',2
c,3,b'3',3
d,4,b'4',4


In [43]:
# Producto de dataframes
dataframe * dataframe1

TypeError: can't multiply sequence by non-int of type 'bytes'

#### Trasposición - Sólo DataFrames

Podemos trasponer filas por columnas, pero únicamente en DataFrame (ya que las series sólo pueden ser unidimensionales). Básicamente lo que se realizará es intercambiar el índice de columnas por el de filas.

In [44]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [45]:
dataframe.T # Conviertimos las filas a columnas

,a,b,c,d
var1,1,2,3,4
var2,b'1',b'2',b'3',b'4'


#### Funciones de numpy (Universal functions, operaciones matemáticas...)

Podemos aplicar cualquier función de NumPy a cualquier estructura de pandas.<br/>

<b>IMPORTANTE:</b> De nuevo, al poder tener múltiples tipos en DataFrames no siempre se podrán aplicar las operaciones (o el resultado obtenido no será el esperado). Además, en el caso de DataFrames en caso de no indicar un valor para <b>axis</b> se aplicará la operación por columnas y nunca sobre el DataFrame completo.

In [46]:
# Operaciones sobre series
np.sqrt(serie)

a    1.000000
b    1.414214
c    1.732051
d    2.000000
dtype: float64

In [47]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [48]:
# Operaciones sobre dataframes (columna a columna, por defecto)
np.sum(dataframe)

var1         10
var2    b'1234'
dtype: object

In [49]:
# Operaciones sobre dataframes (especificando eje)
np.sum(dataframe, axis=1)

TypeError: unsupported operand type(s) for +: 'int' and 'bytes'